# Working on the AI Agent with some community tools

In [ ]:
!pip install langgraph langsmith langchain langchain_groq langchain_community wikipedia arxiv

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
langsmith_api_key = os.getenv("LANGSMITH_API_KEY")

os.environ["LANGCHAIN_API_KEY"] = langsmith_api_key
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "LangGraph_Agent"

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import START, END, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

## Setting up tools

In [ ]:
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=300)
wiki_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=300)

arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_wrapper)
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_wrapper)

### Example of Wiki tool usage

In [ ]:
wiki_tool.invoke("Who is Zeng Laishun?")

In [ ]:
arxiv_tool.invoke("Attention is all you need")

## Building the graph now

In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(api_key=groq_api_key, model="Gemma2-9b-It")

In [ ]:
tools = [arxiv_tool, wiki_tool]

In [ ]:
llm = llm.bind_tools(tools=tools)

In [ ]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [ ]:
def chatbot(state: State) -> str:
    return {"messages": [llm.invoke(state["messages"])]}

In [ ]:
graph_builder = StateGraph(State)

graph_builder.add_node("chatbot", chatbot)
tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

In [ ]:
graph = graph_builder.compile()

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass

In [ ]:
user_inputs = [
    # "What is the latest research on quantum computing?",
    "Who is Zeng Laishun?",
    "What is the paper 'Attention is all you need' about?",
]

for user_input in user_inputs:
    events = graph.stream({"messages": [("user", user_input)]}, stream_mode="values")

    for event in events:
        event["messages"][-1].pretty_print()